In [10]:
# 엑셀 파일의 특정 셀 내용 읽어오기

import openpyxl

wb = openpyxl.load_workbook('./sample.xlsx')
sheet = wb['Sheet1']

print(sheet.max_column, sheet.max_row)
print(sheet.cell(row=1, column=1).value)
print(sheet.cell(row=2, column=1).value)

wb.close()

3 6
이름
Kei


In [8]:
type(wb)

openpyxl.workbook.workbook.Workbook

In [11]:
# 엑셀 파일의 모든 셀 내용 읽어오기

import openpyxl

wb = openpyxl.load_workbook('./sample.xlsx')
sheet = wb['Sheet1']

# 안쪽 열, 바깥쪽 행
for row in sheet.iter_rows(min_row=2): # 행
    for cell in row: # 열
        print(cell.value)
    print('-'*10)
wb.close()

Kei
35
1234-1234
----------
Hong
26
4320-1420
----------
Kim
41
1234-5678
----------
Choi
38
4023-3411
----------
Go
27
1469-4023
----------


In [12]:
# 엑셀 파일에서 지정한 셀 내용 읽어오기
import openpyxl

wb = openpyxl.load_workbook('./sample.xlsx')
sheet = wb['Sheet1']

cells = sheet['A2':'C3']
for row in cells:
    for cell in row:
        print(cell.value)
    print('-'*10)
wb.close()

Kei
35
1234-1234
----------
Hong
26
4320-1420
----------


In [14]:
# openpyxl로 엑셀 파일 쓰기
# 회원정보를 엑셀에 저장

import openpyxl

wb = openpyxl.Workbook()
sheet = wb.active
sheet.title = '회원정보'

# 표 헤더 컬럼 저장
header_titles = ['이름','전화번호']
for idx, title in enumerate(header_titles):
    sheet.cell(row=1, column=idx+1, value=title)
    
# 표 내용 저장
members = [('kei','010-1234-1234'), ('hong','010-4321-1234')]
row_num = 2
for r, member in enumerate(members):
    for c,v in enumerate(member):
        sheet.cell(row=row_num+r, column=c+1, value=v)
        
wb.save('./members.xlsx')
wb.close()

In [18]:
# csv 자료를 테이블에 저장하기
'''
cdv -> db table
    1차 실행 : table 생성 -> 레코드 추가
    2차 실행 : 레코드 검색
'''

import pandas as pd
import pymysql

# (1) csv 파일 로드
bmi = pd.read_csv("data/bmi.csv")
print(bmi.info())

# (2) 각 컬럼 추출
height = bmi['height']
weight = bmi['weight']
label = bmi['label']

config = {
    'host' : '127.0.0.1',
    'user' : 'root',
    'password' : '0000',
    'database' : 'work',
    'port' : 3306,
    'charset' : 'utf8',
    'use_unicode' : True
}

try:
    conn = pymysql.connect(**config)
    cursor = conn.cursor()
    
    # (3) table 조회
    cursor.execute("show tables")
    tables = cursor.fetchall()
    
    # (4) 스위칭기법
    sw = False
    for table in tables:
        if table[0] == 'bmi_tab':
            sw = True # table 있는 경우 swapping
    
    # (5) table 생성
    if not sw :
        print('테이블 없음') # table 없으면 생성
        sql = '''create table bmi_tab(
            height int not null,
            weight int not null,
            label varchar(15) not null)'''
        cursor.execute(sql)
        
    # (6) 레코드 조회
    cursor.execute("select * from bmi_tab")
    rows = cursor.fetchall()
    
    if rows: # (7) 레코드가 있는 경우 : 레코드 조회
        for row in rows:
            print(f"{row[0]}    {row[1]}    {row[2]}")
        print('전체 레코드 수 : ', len(rows))

    else: # (8) 레코드 없는 경우 : 레코드 추가
        print("100 레코드 추가")
        for i in range(100) :
            h = height[i]
            w = weight[i]
            lab = label[i]
            cursor.execute(f"insert into bmi_tab values({h},{w},'{lab}')")
            conn.commit()
except Exception as e :
    print('db error :', e)

finally:
    cursor.close()
    conn.close()    
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   height  20000 non-null  int64 
 1   weight  20000 non-null  int64 
 2   label   20000 non-null  object
dtypes: int64(2), object(1)
memory usage: 468.9+ KB
None
184    61    thin
189    56    thin
183    79    normal
143    40    normal
187    66    normal
161    52    normal
186    54    thin
144    57    fat
184    55    thin
165    76    fat
143    45    normal
154    61    fat
159    57    normal
177    45    thin
174    78    fat
140    80    fat
150    75    fat
141    47    normal
186    57    thin
154    83    fat
141    65    fat
178    74    normal
157    84    fat
175    56    thin
172    40    thin
170    69    normal
143    64    fat
148    68    fat
161    56    normal
176    77    normal
142    60    fat
145    41    normal
179    84    fat
164    80    fat
153    53    normal
182    45    

In [22]:
print(tables)

(('bmi_tab',), ('course',), ('goods',), ('score',), ('stud',))
